In [11]:
import os
import torch
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
from telegram import Update
from transformers import pipeline
import nest_asyncio

# Read API token from the text file
def read_api_token(file_path: str) -> str:
    with open(file_path, 'r') as file:
        return file.read().strip()

# Initialize TinyLlama Model
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
pipe = pipeline(
    "text-generation",
    model=MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Function to generate a response based on user input
def generate_response(user_input: str) -> str:
    """Generate a response using TinyLlama"""

    messages = [
        {"role": "system", "content": "You are a helpful assistant who provides concise and factual responses."},
        {"role": "user", "content": user_input},
    ]
    # Format the input for the TinyLlama model
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    # Return the generated text response
    return outputs[0]["generated_text"].split("<|assistant|>")[-1].strip()

# Function to handle the /start command
async def start(update: Update, context: CallbackContext) -> None:
    await update.message.reply_text("Hi! This is your AI Assistant bot. How can I help you?")

# Function to process the user’s message and generate a response
async def process(update: Update, context: CallbackContext) -> None:
    user_message = update.message.text

    # Query the model for a response
    try:
        response = generate_response(user_message)
    except Exception as e:
        response = "There was an error processing your request. Please try again later."
        print(f"Error: {e}")
    await update.message.reply_text(response)

def main() -> None:
    """Start the bot"""
    # Read the API token from the file
    API_TOKEN = read_api_token("API.txt")
    application = Application.builder().token(API_TOKEN).build()

    # Add command and message handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, process))

    # Start polling for updates
    application.run_polling()

if __name__ == "__main__":
    main()


Device set to use cuda:0


RuntimeError: Cannot close a running event loop